In [134]:
# head cell
import json
import pandas as pd
import numpy as np

with open('mhrice.json') as file:
    raw = json.load(file)

### Monster Names & IDs
SECTIONS:
- **monster_names**: rise english names
- **monster_names_mr**: sunbreak english names
- **monsters**: IDs
- **random_mystery_enemy**: monster A★ and M★ rank numbers

COLUMNS:
- **id**: base monster id
- **sub_id**: indicates monster variant
- **em_id** (enemy_type): monster id when referenced by "EnemyIndex###(_MR)"
- **em_type.Em**: internal monster identifier
- **mon_name**: monster name
- **normal_rank**: M★ rank. 12 indicates access through anomaly only.
- **mystery_rank**: A★ rank. 12 indicates access as secondary target only.

OUTPUT: **df_monsters**
| id | sub_id | em_id | em_type.Em | mon_name | normal_rank | mystery_rank |
|-|-|-|-|-|-|-|
| 1 | 0 | 0 | 1 | Rathian | 2 | 3 |
| 1 | 2 | 76 | 513 | Gold Rathian | 6 | 7 |
| 1 | 7 | 1 | 1793 | Apex Rathian | 12 | 12 |
| 2 | 0 | 2 | 2 | Rathalos | 4 | 5 |
| 2 | 2 | 77 | 514 | Silver Rathalos | 6 | 7 |

In [79]:
manual_entry = True #maintaints Toadversary and non-anomaly MR monsters in data

# Monster Names
sect = raw.get('monster_names').get('entries')
df = pd.json_normalize(sect)
df['mon_name'] = df['content'].apply(lambda x: x[1]) #grab only english name
df['em_id'] = pd.to_numeric(df['name'].apply(lambda x: x[-3:])) #grab only id number
df = df.iloc[0:46] #remove small monsters
df_monster_names = df[['em_id', 'mon_name']].copy()

# MR Monster Names
sect = raw.get('monster_names_mr').get('entries')
df = pd.json_normalize(sect)
df['mon_name'] = df['content'].apply(lambda x: x[1])
df['em_id'] = pd.to_numeric(df['name'].apply(lambda x: x[-6:-3]))
df = pd.concat([df.iloc[0:23], df.iloc[31:]], axis=0, ignore_index=True)

df_monster_names = pd.concat([df_monster_names, df[['em_id', 'mon_name']]], axis=0, ignore_index=True)

# Monster IDs
sect = raw.get('monsters')
df = pd.json_normalize(sect)
df = df[['id','sub_id','enemy_type','em_type.Em']]
df.rename(columns={'enemy_type':'em_id'}, inplace=True)

df_monsters = pd.merge(df,df_monster_names, on='em_id', how='left')

if manual_entry:
    df_monsters.loc[df_monsters['em_id'] == 46, 'mon_name'] = 'Toadversary' # add Toadversary to list
else:
    df_monsters.dropna(subset=['mon_name'], inplace=True) # drop Toadversary from list
    df_monsters.reset_index(drop=True, inplace=True)

# Anomaly Data
sect = raw.get('random_mystery_enemy').get('lot_enemy_list')
df = pd.json_normalize(sect)
df = df[['em_type.Em','normal_rank','mystery_rank']]

df_monsters = pd.merge(df_monsters,df, on='em_type.Em', how='left')

if manual_entry:
    df_monsters.loc[df_monsters['mon_name'] == 'Amatsu', 'normal_rank'] = 6
    df_monsters.loc[df_monsters['mon_name'] == 'Wind Serpent Ibushi', 'normal_rank'] = 6
    df_monsters.loc[df_monsters['mon_name'] == 'Thunder Serpent Narwa', 'normal_rank'] = -1
    df_monsters.loc[df_monsters['mon_name'] == 'Narwa the Allmother', 'normal_rank'] = 6
    df_monsters.loc[df_monsters['mon_name'] == 'Gaismagorm', 'normal_rank'] = 6
else:
    df_monsters.dropna(subset=['mystery_rank'], inplace=True)
    df_monsters.reset_index(drop=True, inplace=True)

df_monsters.fillna(-1, inplace=True)
df_monsters[['normal_rank','mystery_rank']] = df_monsters[['normal_rank','mystery_rank']].astype(int)

# df_monsters.to_excel('monster.xlsx')
# print(df_monsters[0:5])

### Item Names, IDs, & Properties
SECTIONS:
- **items_name_msg**: rise english names & ids
- **items_name_msg_mr**: sunbreak english names & ids
- **items**: item properties

OUTPUT: **df_items**
| item_id | item_name | type_ | category_prefix | material_category | category_worth | rare |
|-|-|-|-|-|-|-|
| 6 | Potion | Consume | 'None' | [] | 0 | 1 |
| 7 | Mega Potion | Consume | 'None' | [] | 0 | 2 |
| 5 | Max Potion | Consume | 'None' | [] | 0 | 3 |
| 467 | Ancient Potion | Consume | 'None' | [] | 0 | 4 |
| 13 | Antidote | Consume | 'None' | [] | 0 | 1 |

In [2]:
# Item Names & IDs
sect = raw.get('items_name_msg').get('entries')
df = pd.json_normalize(sect)
df.rename(columns={'name':'item_id'}, inplace=True)
df=df[~df['item_id'].str.contains('I_EC_')] # remove endemic life
df['item_id'] = df['item_id'].str.extract('(\d+)', expand=False) # grab only item number
df.dropna(inplace=True) # drop blanks
df['item_id'] = pd.to_numeric(df['item_id'], errors='coerce', downcast='integer') # convert column to int
df['item_name'] = df['content'].apply(lambda x: x[1]) # grab only english name
df = df[~df['item_name'].str.contains('COLOR FF0000', na=False)] # remove placeholder items
df_item_names = df[['item_id', 'item_name']].copy()

# MR Item Names & IDs
sect = raw.get('items_name_msg_mr').get('entries')
df = pd.json_normalize(sect)
df.rename(columns={'name':'item_id'}, inplace=True)
df=df[~df['item_id'].str.contains('I_EC_')]
df['item_id'] = df['item_id'].str.extract('(\d+)', expand=False)
df.dropna(inplace=True)
df['item_id'] = pd.to_numeric(df['item_id'], errors='coerce', downcast='integer')
df['item_name'] = df['content'].apply(lambda x: x[1])
df = df[~df['item_name'].str.contains('COLOR FF0000', na=False)]

df_item_names = pd.concat((df_item_names, df[['item_id', 'item_name']]), axis=0, ignore_index=True)

# Item Types, Categories, Worth, and Rarity
sect = raw.get('items').get('param')
df = pd.json_normalize(sect)
df.rename(columns={'id.Normal':'item_id'}, inplace=True)
df = df[['item_id', 'type_', 'material_category', 'category_worth', 'rare']]
df = pd.merge(df, df_item_names, on='item_id', how='left')
df.dropna(subset=['item_name'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Denesting Category Dictionary
df['category_prefix'] = df['material_category'].apply(lambda x: next(iter(x[0])) if isinstance(x[0],dict) else x[0]) # prefix
df['material_category'] = df['material_category'].apply(lambda x: [d for d in x if isinstance(d, dict)]) # None removal
df['material_category'] = df['material_category'].apply(lambda x: [list(d.values())[0] for d in x] if x else []) # denest

df_items = df[['item_id', 'item_name', 'type_', 'category_prefix', 'material_category', 'category_worth', 'rare']].copy()

# print(df_items.head(5))
# df_items.to_excel('items.xlsx')

### Drop Tables
SECTIONS:
- **reward_id_lot_table**: base game item drop chance and count tables
- **reward_id_lot_table_mr**: sunbreak item drop chance and count tables

OUTPUT: **df_drop_tables**
| drop_table_id | lot_rule | item_id_list | num_list | probability_list |
|-|-|-|-|-|
| 310000 | Random | [125, 126, 687, 128, 699, 131, 719] | [1, 1, 1, 2, 1, 1, 1] | [17, 24, 15, 22, 8, 8, 6] |
| 310001 | Random | [691, 132, 720, 705, 704, 129, 700] | [1, 1, 1, 1, 1, 1, 1] | [9, 14, 7, 19, 25, 14, 12] |
| 310002 | Random | [693, 133, 710, 706, 707, 688, 701] | [1, 1, 1, 1, 1, 1, 1] | [5, 11, 16, 18, 14, 21, 15] |
| 310003 | Random | [702, 484, 686, 689, 127, 133, 721] | [2, 1, 1, 1, 1, 1, 1] | [18, 17, 23, 14, 9, 13, 6] |
| 310004 | Random | [695, 134, 479, 711, 690, 718, 127] | [1, 1, 1, 1, 1, 1, 1] | [5, 9, 19, 19, 14, 23, 11] |

In [3]:
# Item Drop Tables
sect = raw.get('reward_id_lot_table').get('param')
df = pd.json_normalize(sect)
df['item_id_list'] = df['item_id_list'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
df['num_list'] = df['num_list'].apply(lambda y: [x for x in y if x>0])
df['probability_list'] = df['probability_list'].apply(lambda y: [x for x in y if x>0])
df_drop_tables = df.copy()

# MR Item Drop Tables
sect = raw.get('reward_id_lot_table_mr').get('param')
df = pd.json_normalize(sect)
df['item_id_list'] = df['item_id_list'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
df['num_list'] = df['num_list'].apply(lambda y: [x for x in y if x>0])
df['probability_list'] = df['probability_list'].apply(lambda y: [x for x in y if x>0])
df_drop_tables = pd.concat([df_drop_tables, df], axis=0, ignore_index=True)
df_drop_tables.rename(columns={'id':'drop_table_id'}, inplace=True)

# df_drop_tables.to_excel('item_drop_tables.xlsx')
# print(df_drop_tables)

### Anomaly Items, Subset of Items

COLUMNS:
- **item_id**: item ID
- **item_name**: item name
- **type**: material type [aff, ess, coin]
- **category_worth**: item value in augmenting and charm melding

OUTPUT: **df_anomaly_items**
| item_id | item_name | type | category_worth |
|-|-|-|-|
| 2521 | Amber Essence | ess | 10 |
| 2522 | Amber Essence+ | ess | 20 |
| 2523 | Prime Amber Essence | ess | 40 |
| 2584 | Afflicted Bone | aff | 2 |
| 2585| Afflicted Pelt | aff | 2 |

In [ ]:
df = df_items[~df_items['category_prefix'].str.contains('LrHr')] #remove LrHr items so that only Mr or None results show
df_anomaly_items = df[df['material_category'].apply(lambda x: 73 in x)].copy()
df_anomaly_items['type'] = 'aff'
df_ess = df[df['material_category'].apply(lambda x: 74 in x)].copy()
df_ess['type'] = 'ess'
df_coin = df_items[df_items['item_name'].str.contains('Investigation Coin',case=False,na=False)].copy()
df_coin['type'] = 'coin'
df_anomaly_items = pd.concat([df_anomaly_items, df_ess, df_coin], axis=0, ignore_index=True)
df_anomaly_items = df_anomaly_items[['item_id','item_name','type','category_worth']]
df_anomaly_items.reset_index(drop=True, inplace=True)

# df_anomaly_items.to_excel('anom_items.xlsx')
# print(df_anomaly_items)

### Anomaly Drop Lots
SECTIONS:
- **mystery_reward_item**: anomaly drop lots for quests, investigations, and special investigations

COLUMNS:
- **lv_lower** (~~lv_lower_limit~~): Anomaly investigation reward bracket lower level. 0 for quests. 300 for special investigations.
- **lv_upper** (~~lv_upper_limit~~): Anomaly investigation reward bracket upper level. 0 for quests. 300 for special investigations.
- **em_type.Em**: primary target monster internal ID
- **item_id** (~~reward_item.Normal~~): the primary afflicted material dropped by primary target monster on carve and part break
- **anom_item_chance** (~~hagibui_probability~~): chance of getting primary afflicted material on carve and part break
- **anom_drop_table_id** (~~quest_reward_table_index~~): table ID for quest rewards.
- **anom_drop_lot_ids** (~~additional_quest_reward_table_index~~): additional quest reward table IDs.
- **sp_ess_table_id** (~~special_quest_reward_table_index~~): bonus essence drop table for single AR brackets (e.g. AR200, AR220)
- **multi_ess_table_id** (~~multiple_target_reward_table_index~~): bonus essence drop table for hunting monster as secondary target
- **multi_coin_table_id**: (~~multiple_fix_reward_table_index~~): bonus coin drop table for hunting monster as secondary target
- **mystery_reward_table**: ???

OUTPUT: **df_anom_drop_lots**
| lv_lower | lv_upper | anom_item_chance | anom_drop_table_id | anom_drop_lot_ids |
|-|-|-|-|-|
| 0 | 0 | 40 | 0 | [0, 0, 0, 0] |
| 1 | 30 | 40 | 560000 | [550000, 540000, 0, 330300] |
| 31 | 50 | 40 | 560000 | [550100, 540001, 550000, 330300] |
| 51 | 70 | 40 | 560001 | [550100, 540100, 550000, 330300] |
| 71 | 90 | 40 | 560001 | [550101, 540100, 550000, 330301] |

| sp_ess_table_id | multi_ess_table_id | multi_coin_table_id | em_type.Em | item_id |
|-|-|-|-|-|
| 0 | 0 | 0 | 98 | 2585 |
| 0 | 760000 | 770000 | 98 | 2585 |
| 0 | 760000 | 770000 | 98 | 2885 |
| 0 | 760001 | 770001 | 98 | 2885 |
| 0 | 760001 | 770001 | 98 | 2885 |

In [104]:
# Anomaly Drop Lots
sect = raw.get('mystery_reward_item').get('param')
df = pd.json_normalize(sect)
df = df[df['is_special_mystery']==False] # remove SI entries since drops are the same as AR300 and represented separately in [300,300] investigations
df = df[~((df['lv_lower_limit']==0) & (df['lv_upper_limit']==0))] # remove 0-0 entries as they don't represent anomaly quest rewards

df['reward_item.Normal'] = pd.to_numeric(df['reward_item.Normal'].fillna(-1), errors='ignore', downcast='integer')
df.drop(['is_special_mystery','quest_no', 'item_num', 'reward_item', 'mystery_reward_table'], axis=1, inplace=True)

df.rename(columns={'lv_lower_limit':'lv_lower',
           'lv_upper_limit':'lv_upper',
           'reward_item.Normal':'item_id',
           'hagibui_probability':'anom_item_chance',
           'quest_reward_table_index':'anom_drop_table_id',
           'additional_quest_reward_table_index':'anom_drop_lot_ids',
           'special_quest_reward_table_index':'sp_ess_table_id',
           'multiple_target_reward_table_index':'multi_ess_table_id',
           'multiple_fix_reward_table_index':'multi_coin_table_id'
           },inplace=True)
df.reset_index(drop=True, inplace=True)

df_anom_drop_lots = df.copy()
# print(df_anom_drop_lots)

### Anomaly Material Average Item Drop

REFERENCES:
- **df_anomaly_items**
- **df_drop_tables**

COLUMNS:
- **drop_table_id**: drop table ID
- **aff_avg**: average afflicted material melding worth
- **ess_avg**: average essence augmenting worth
- **coin_avg**: average investigation coin worth

OUTPUT: **df_anom_drop_avg**
| drop_table_id | aff_avg | ess_avg | coin_avg |
|-|-|-|-|
| 530000 | 4.8 | 0.0 | 0.0 |
| 530001 | 4.8 | 0.0 | 0.0 |
| 530002 | 9.6 | 0.0 | 0.0 |
| 530003 | 9.6 | 0.0 | 0.0 |
| 530004 | 14.4 | 0.0 | 0.0 |

In [ ]:
#filter out non-anomaly drop tables
mapper = set(df_anomaly_items['item_id'])
df = df_drop_tables[df_drop_tables['item_id_list'].apply(lambda x: any(y in mapper for y in x))]

#add translated names to items
# mapper = dict(zip(df_items['item_id'], df_items['item_name']))
# df_drop_tables['tab']=df_drop_tables['item_id_list'].apply(lambda x: [mapper.get(i,'?') for i in x])

# translate item_id_list into worth_list
df2 = df_anomaly_items[df_anomaly_items['type']=='aff']
mapper = dict(zip(df2['item_id'], df2['category_worth']))
df['aff_worth_list'] = df['item_id_list'].apply(lambda x: [mapper.get(i,0) for i in x])
df2 = df_anomaly_items[df_anomaly_items['type']=='ess']
mapper = dict(zip(df2['item_id'], df2['category_worth']))
df['ess_worth_list'] = df['item_id_list'].apply(lambda x: [mapper.get(i,0) for i in x])
df2 = df_anomaly_items[df_anomaly_items['type']=='coin']
mapper = dict(zip(df2['item_id'], [1]))
df['coin_worth_list'] = df['item_id_list'].apply(lambda x: [mapper.get(i,0) for i in x])

# sum of products: item_cts, item_chance, aff_worth
df['aff_avg'] = df.apply(lambda x: round(np.sum(np.array(x['num_list']) * np.array(x['probability_list']) * np.array(x['aff_worth_list']) /100),2), axis=1)
df['ess_avg'] = df.apply(lambda x: round(np.sum(np.array(x['num_list']) * np.array(x['probability_list']) * np.array(x['ess_worth_list']) /100),2), axis=1)
df['coin_avg'] = df.apply(lambda x: round(np.sum(np.array(x['num_list']) * np.array(x['probability_list']) * np.array(x['coin_worth_list']) /100),2), axis=1)

df_anom_drop_avg = df[['drop_table_id','aff_avg','ess_avg','coin_avg']].copy()

# df_anom_drop_avg['am_avg'] = df_anom_drop_avg.apply(lambda x: [x['aff_avg'], x['ess_avg'], x['coin_avg']], axis=1)
# df_anom_drop_avg = df[['drop_table_id','am_avg']]

# df_anom_drop_avg.to_excel('anom_drop_avg.xlsx')
# print(df_anom_drop_avg)

### Average Material Drop Per Anomaly Investigation Level Bracket
REFERENCES:
- **df_anom_drop_lots**
- **df_anomaly_items**
- **df_anom_drop_avg**
- **df_monsters**

COLUMNS:
- **lv_lower**: anomaly investigation level bracket lower end
- **lv_upper**: anomaly investigation level bracket upper end
- **item_name**: primary afflicted material reward
- **aff_avg**: average afflicted material augment and melding value
- **ess_avg**: average essence augment value
- **coin_avg**: average investigation coin value
- **ess_avg_multi**: average essence augment value for multi-monster investigations
- **coin_avg_multi**: average investigation coin value for multi-monster investigations
- **mon_name**: primary target name
- **mystery_rank**: primary target rank

OUTPUT: **df_anom_avg**
| lv_lower | lv_upper | item_name | aff_avg | ess_avg | coin_avg | ess_avg_multi | coin_avg_multi | mon_name | mystery_rank |
|-|-|-|-|-|-|-|-|-|-|
| 1 | 30 | Afflicted Pelt | 7.2 | 56.5 | 0.05 | 29 | 2 | Great Izuchi | 1 |
| 31 | 50 | Afflicted Hide+ | 29.6 | 67.5 | 0.05 | 29 | 2 | Great Izuchi | 1 |
| 51 | 70 | Afflicted Hide+ | 29.6 | 115 | 0.25 | 58 | 2.4 | Great Izuchi | 1 |
| 71 | 90 | Afflicted Hide+ | 38.4 | 115 | 0.25 | 58 | 2.4 | Great Izuchi | 1 |
| 91 | 100 | Afflicted Hide+ | 63.8 | 289 | 0.25 | 84 | 2.4 | Great Izuchi | 1 |


In [ ]:
carves = 4 # number of carves + estimated partbreaks
df = df_anom_drop_lots

#replace item_id with item name and worth from df_anomaly_items
df = pd.merge(df, df_anomaly_items[['item_id','item_name','category_worth']], on='item_id', how='left')
df.drop(columns=['item_id'], inplace=True)
df.fillna(0,inplace=True)

# #create carve reward worth column
df['carve_worth'] = df.apply(lambda x: round(x['category_worth']*x['anom_item_chance']*carves/100,2), axis=1)
df.drop(columns=['category_worth','anom_item_chance'], inplace=True)

# #replace reward tables with am, ess, coin worth averages
df = pd.merge(left=df, left_on='anom_drop_table_id', right=df_anom_drop_avg, right_on='drop_table_id', how='left')
df.drop(columns=['anom_drop_table_id','drop_table_id'], inplace=True)
df.fillna(0,inplace=True)
# df = df.rename(columns={'am_avg':'quest_rewards'})
df['aff_avg'] = df['aff_avg']+df['carve_worth']
df.drop(columns='carve_worth', inplace=True)

#replace additional quest rewards with am worth averages
mapper = dict(zip(df_anom_drop_avg['drop_table_id'], list(zip(df_anom_drop_avg['aff_avg'],df_anom_drop_avg['ess_avg'],df_anom_drop_avg['coin_avg']))))
df['aqr'] = df['anom_drop_lot_ids'].apply(lambda x: [mapper.get(i,[0,0,0]) for i in x])
#zip addition quest rewards
df['aqr_sum'] = df['aqr'].apply(lambda x: [round(sum(i),2) for i in zip(*x)])
df.drop(columns=['aqr','anom_drop_lot_ids'], inplace=True)
#individual column sum
df['aff_avg'] = df.apply(lambda x: x['aff_avg']+x['aqr_sum'][0], axis=1)
df['ess_avg'] = df.apply(lambda x: x['ess_avg']+x['aqr_sum'][1], axis=1)
df['coin_avg'] = df.apply(lambda x: x['coin_avg']+x['aqr_sum'][2], axis=1)
df.drop(columns='aqr_sum', inplace=True)

#sp essence rewards
df = pd.merge(left=df, left_on='sp_ess_table_id', right=df_anom_drop_avg[['drop_table_id','ess_avg']], right_on='drop_table_id', how='left', suffixes=['','_sp'])
df.fillna(0,inplace=True)
df['ess_avg'] = df['ess_avg']+df['ess_avg_sp']
df.drop(columns=['drop_table_id','ess_avg_sp','sp_ess_table_id'], inplace=True)

#multi rewards
df = pd.merge(left=df, left_on='multi_ess_table_id', right=df_anom_drop_avg[['drop_table_id','ess_avg']], right_on='drop_table_id', how='left', suffixes=['','_multi'])
df.fillna(0,inplace=True)
df.drop(columns=['multi_ess_table_id','drop_table_id'], inplace=True)
df = pd.merge(left=df, left_on='multi_coin_table_id', right=df_anom_drop_avg[['drop_table_id','coin_avg']], right_on='drop_table_id', how='left', suffixes=['','_multi'])
df.fillna(0,inplace=True)
df.drop(columns=['multi_coin_table_id','drop_table_id'], inplace=True)

#monster names
df = pd.merge(df, df_monsters[['em_type.Em','mon_name','mystery_rank']], on='em_type.Em',how='left')
df.drop(columns=['em_type.Em'], inplace=True)

df_anom_avg = df.copy()
# df_anom_avg.to_excel('anom_avg.xlsx')
# print(df_anom_avg)

### Armor Skill Name & ID

player_skill_name_msg

In [ ]:
# Skill Names & IDs
sect = raw.get('player_skill_name_msg').get('entries')
df = pd.json_normalize(sect)
df['skill_name'] = df['content'].apply(lambda x: x[1]) #grab only english name
df['skill_id'] = df['name'].str.extract('(\d+)', expand=False) # grab id numbers
df = df[df['skill_name']!=''] # remove blank names
df.dropna(inplace=True) # remove blank ids
df['skill_id'] = pd.to_numeric(df['skill_id'], errors='ignore', downcast='integer')
df.reset_index(drop=True, inplace=True)
df_skill_names = df[['skill_id', 'skill_name']].copy()

# MR Skill Names & IDs
sect = raw.get('player_skill_name_msg_mr').get('entries')
df = pd.json_normalize(sect)
df['skill_name'] = df['content'].apply(lambda x: x[1]) #grab only english name
df['skill_id'] = df['name'].str.extract('(\d+)', expand=False) # grab id numbers
df = df[df['skill_name']!=''] # remove blank names
df.dropna(inplace=True) # remove blank ids
df['skill_id'] = pd.to_numeric(df['skill_id'], errors='ignore', downcast='integer')
df.reset_index(drop=True, inplace=True)
df_skill_names = pd.concat([df_skill_names, df[['skill_id', 'skill_name']]], axis=0, ignore_index=True)

# print(df_skill_names)

## Decorations
### "decorations": param
- **decorations**: "id":{"Deco": 11}
- **decoration_lv**: 2
- **skill_id_list**: [{"Skill": 11},"None"]
- **skill_lv_list**: [1,0]

### "decorations_product": param
- **id**: {"Deco": 0}
- **item_flag**: "None"
- **enemy_flag**: {"Em": 99}
- **progress_flag**: 0
- **item_id_list**: [{"Normal": 1036},
                     {"Normal": 337},
                     {"Normal": 571},
                     {"Normal": 179}]
- **item_num_list**: [5,3,3,1]

### "decorations_name_msg":entries
- **name**: "Decorations_000_Name
- **content**: ["...", "Attack Jewel 2", "..."]

### "decorations_name_msg_mr":entries
- **name**: "Decorations_201_Name
- **content**: ["...", "Hard Fire Res Jewel 4", "..."]

In [95]:
# Decoration Names
sect = raw.get('decorations_name_msg').get('entries')
df = pd.json_normalize(sect)
df['deco_name'] = df['content'].apply(lambda x: x[1]) #grab only english name
df['deco_id'] = df['name'].str.extract('(\d+)', expand=False) # grab id numbers
df = df[df['deco_name']!=''] # remove blank names
df.dropna(inplace=True) # remove blank ids
df['deco_id'] = pd.to_numeric(df['deco_id'], errors='ignore', downcast='integer')
df.reset_index(drop=True, inplace=True)
df_deco_names = df[['deco_id', 'deco_name']].copy()

# MR Decoration Names
sect = raw.get('decorations_name_msg_mr').get('entries')
df = pd.json_normalize(sect)
df['deco_name'] = df['content'].apply(lambda x: x[1]) #grab only english name
df['deco_id'] = df['name'].str.extract('(\d+)', expand=False) # grab id numbers
df = df[df['deco_name']!=''] # remove blank names
df = df[~df['deco_name'].str.contains('COLOR FF0000', na=False)]
df.dropna(inplace=True) # remove blank ids
df['deco_id'] = pd.to_numeric(df['deco_id'], errors='ignore', downcast='integer')
df.reset_index(drop=True, inplace=True)
df_deco_names = pd.concat([df_deco_names, df[['deco_id', 'deco_name']]], axis=0, ignore_index=True)

# Decoration Skills
sect = raw.get('decorations').get('param')
df = pd.json_normalize(sect)
df['skill_grade'] = df['skill_id_list'].apply(lambda x: next(iter(x[0]))) # grab MR category
df['skill_id'] = df['skill_id_list'].apply(lambda x: next(iter(x[0].values()))) # grab skill ID
df.loc[df['skill_grade']=='MrSkill', 'skill_id'] += 200 # convert skill ID to +200 for MR
df['id.Deco'] = df['id.Deco'].fillna(df['id.MrDeco']+200) # merge MrDeco ID into Deco IDs
df.rename(columns={'id.Deco':'deco_id',
                   'decoration_lv':'deco_size'}, inplace=True)
df_deco_skills = df[['deco_id','deco_size','skill_id']].copy()

# Decoration Unlock Conditions
sect = raw.get('decorations_product').get('param')
df = pd.json_normalize(sect)
df['item_id_list'] = df['item_id_list'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
df['item_num_list'] = df['item_num_list'].apply(lambda y: [x for x in y if x>0])
df['id.Deco'] = df['id.Deco'].fillna(df['id.MrDeco']+200) # merge MrDeco ID into Deco IDs
df['item_flag.Normal'] = df['item_flag.Normal'].fillna(-1)
df.rename(columns={'id.Deco':'deco_id',
                   'enemy_flag.Em':'key_em_type.Em',
                   'item_flag.Normal':'key_item_id',
                   'item_id_list':'craft_item_id_list',
                   'item_num_list':'craft_num_list'}, inplace=True)
df_deco_flags = df[['deco_id','craft_item_id_list','craft_num_list','progress_flag','key_em_type.Em','key_item_id']].copy()
df_deco_flags['deco_id'] = pd.to_numeric(df_deco_flags['deco_id'], errors='ignore', downcast='integer')
df_deco_flags['key_item_id'] = pd.to_numeric(df_deco_flags['key_item_id'], errors='ignore', downcast='integer')




# Bring it together
# df_decos = pd.merge()

# print(df_deco_flags)
# print(df_decos)

In [96]:
# Progress Flags
sect = raw.get('progress').get('param_list')
df = pd.json_normalize(sect)
df.fillna(-1)
df_progress_flags = df[['progress_flag',
                        'quest_no',
                        'enable_progress_hr_check',
                        'progress_hr',
                        'village.VillageProgress',
                        'hall.HallProgress',
                        'mr.MasterRankProgress']].copy() # currently ignoring talk flags

print(df_progress_flags)

In [137]:
# Quest Lists
q_sects = ['quest_arena_msg',
           'quest_dlc_msg',
           'quest_hall_msg',
           'quest_hall_msg_mr',
           'quest_hall_msg_mr2',
           'quest_tutorial_msg',
           'quest_village_msg',
           'npc_mission_msg',
           'npc_mission_msg_mr']
df_quest_names = pd.DataFrame()
for q in q_sects:
    sect = raw.get(q).get('entries')
    df = pd.json_normalize(sect)
    df = df[df['name'].str.contains('_01')]
    df['quest_type'] = df['name'].str.extract('(^[A-Za-z]+)')
    df['quest_id'] = df['name'].str.extract('(\d+)')
    df['quest_id'] = pd.to_numeric(df['quest_id'],errors='coerce',downcast='integer')
    df['content'] = df['content'].apply(lambda x: x[1])
    df = df[~df['content'].str.contains('COLOR FF0000', na=False)]
    df_quest_names = pd.concat([df_quest_names,df[['quest_id','quest_type','content']]], axis=0,ignore_index=True)
df_quest_names = df_quest_names[df_quest_names['quest_type']!='QNmystery']
# print(df_quest_names)

In [97]:
df = df_deco_flags
df = pd.merge(left=df, left_on='progress_flag', right=df_progress_flags, right_on='progress_flag', how='left')


In [ ]:
# '''Gathering Node Drop Tables'''
# sect = raw.get('item_pop_lot').get('param')
# df = pd.json_normalize(sect)
# df['lower_item_ids'] = df['lower_id'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
# df['lower_item_ct'] = df['lower_num'].apply(lambda y: [x for x in y if x>0])
# df['lower_item_chance'] = df['lower_probability'].apply(lambda y: [x for x in y if x>0])
# df['upper_item_ids'] = df['upper_id'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
# df['upper_item_ct'] = df['upper_num'].apply(lambda y: [x for x in y if x>0])
# df['upper_item_chance'] = df['upper_probability'].apply(lambda y: [x for x in y if x>0])
# df['master_item_ids'] = df['master_id'].apply(lambda y: [x["Normal"] for x in y if isinstance(x,dict)])
# df['master_item_ct'] = df['master_num'].apply(lambda y: [x for x in y if x>0])
# df['master_item_chance'] = df['master_probability'].apply(lambda y: [x for x in y if x>0])
# df_item_pop_lot = df[['pop_id','field_type','lot_count','lower_item_ids','lower_item_ct', 'lower_item_chance','upper_item_ids','upper_item_ct', 'upper_item_chance','master_item_ids','master_item_ct', 'master_item_chance']]
# print(df_item_pop_lot.head(5))
# df_item_pop_lot.to_excel('gathering_node_drop_lot.xlsx')